[go here](#here)

# [NSF's OIR Lab](https://nationalastro.org/) Astro Data Archive: Advanced Search


[Community Science and Data Center](https://nationalastro.org/programs/csdc/)

### Table Of Contents:
* [Goals & notebook summary](#goals)
* [Summary of this API (list of web services)](#summary)
* [Imports and setup](#init)
* [Details](#details); Example of every service. Special cases. Error Handling
  * [Get Core File fields](#core_file)
  * [Get Aux File fields](#aux_file)
  * [Search Files](#file_search)
      
  * [Get Core HDU fields](#core_hdu)
  * [Get Aux HDU fields](#aux_hdu)
  * [Search HDUs](#hdu_search)
  
* [What if something goes wrong](#errors) (errors)

* [Resources and references](#doc)
* [Glossary](#glos)

## Still needed
- define our special terms (ADD MORE), keep glossary out of the way at end
  + Proctype
- search services should say: No Results, Searched N (File or HDU) records of total available of N Files/HDUs
- provide convenience function in notebook for pretty Error display
- Examples of all output formats (json,xml,csv) for each web-service type that sports "format"
- display file found in search (see sia.ipynb)
- **Services to get the values of each categorical**: proc_type, prod_type, obs_type, obs_mode
- Doc all "modifiers" used in search spec that say how to match value to field (eg. val_overlap)

<a class="anchor" id="goals"></a>
# Goals
Instructions and examples for using Advanced Search API for accessing OIRLab Archive metadata.

# Notebook Summary
We show you how to use the API to search the Archive for FITS files by **any** field in the FITS headers.

Most of the _*search/_ service calls are made with a _limit_ parameter which reduces the amount of output and may speed up the search.  If you don't think you are getting all your results, make sure you have removed the limit parameter!

#### Add "about the current archive"; give various stats (number of files, hdus in archive; period of time covered; most recently added; list of telescopes/insruments!!!

## Summary of this API (list of web services) <a id="summary">
For full list, see: [http://marsnat2.pat.dm.noao.edu:8000/api-docs/#/api](http://marsnat2.pat.dm.noao.edu:8000/api-docs/#/api)

1. [GET] `{ADS}/aux_file_fields/{instrument}/{proctype}/`
   Get fieldnames available from FitsFiles.
3. [GET] `{ADS}/aux_hdu_fields/{instrument}/{proctype}/`
   Get fieldnames available from HDUs.
5. [GET] `{ADS}/core_file_fields/`
7. [GET] `{ADS}/core_hdu_fields/`
9. [POST] `{ADS}/fasearch/`
11. [POST] `{ADS}/hasearch/`

Services of form `core_<type>_fields` (where <type> is one of: "file","hdu") return the list of optimized fields you can search against. 

Services of form `aux_<type>_fields` (where <type> is one of: "file","hdu") are used to return ALL of the FITS header field names that are not alreay included in the "core" lists.  These services require a specific Instrument and Proc_type [define!!!] be given. These fields have no search optimizations so using any of these will slow down your searches.

Services of form `fasearch` or `hasearch` (File/Hdu Advanced SEARCH) perform a search against full FITS files. They are accessed via a  HTTP POST (link) that takes a search specification in its JSON payload.


## Imports and setup <a id="init">
Some of this is just for presentation in the juypter notebook.  You may not need it all.

In [6]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import astropy.io.fits as pyfits
import astropy.utils as autils
import requests
import json
from pprint import pprint as pp  # pretty print

print(f'Using Python version: {sys.version}\n')
with open ("requirements.txt", "r") as f:
    data=f.readlines()
print(f'This Jupyter Notebook uses (some of) these packages:\n\n{"".join(data)}')

Using Python version: 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]

This Jupyter Notebook uses (some of) these packages:

pandas==0.23.4
matplotlib==3.0.2
ipython==7.2.0
jupyter==1.0.0
requests==2.22.0
astropy==3.2.1



In [2]:
# UTILS are settings for HOST etc. that you won't need to use the API
# but are used here to make this notebook more portable.
%run utils.ipynb
tic() # start tracking time of Notebook load

adsurl = f'{natroot}/api/adv_search'
adsurl

'https://marsnat1.stage.dm.noao.edu/api/adv_search'

## Details <a id="details">
    
* [Get Core File fields](#core_file)
* [Get Aux File fields](#aux_file)
* [Search Files](#file_search)
  
* [Get Core HDU fields](#core_hdu)
* [Get Aux HDU fields](#aux_hdu)
* [Search HDUs](#hdu_search)
  
* [What if something goes wrong](#errors) (errors)

Notes: 
- `File` fields and searches refer the Primary HDU of a FITS file. (the field names may be a little different from how they appear in the FITS header since not all FITS headers use the same keywords for the same thing.)  

- `HDU` fields and searches refer to non-primary HDUs of a FITS file. There are up to 70 times more HDU records as File records (depending on the source of the FITS file).

- All of these web services accept a `format` parameter in the HTTP query string.  This defines the format of the output. I can be one of: json (the default), csv, xml

- For the sake readability in this notebook, output is often stored in a Pandas DataFrame for display.

- Searches in this notebook use a `limit` parameter to reduce the number of records returned.

- Strings accept search modifiers: _exact, iexact, startswith, istartswith, endswith, iendswith, contains, icontains, regex, iregex_ (as described in [Django QuerySet API](https://docs.djangoproject.com/en/2.2/ref/models/querysets/)

## Get Core and Aux fields for Files and HDUs
Retrieve list of fields (with data-types) to use later in specifying which fields to filter and which ones to return in the filtered results.
### Get Core File field names and types <a id="core_file" />
These occur once per FITS file and are optimized for fast search. Output is field-name, data-type pairs.  The data-types are intended to be used with Pandas. Note that "catagory" types refer to relatively small sets of acceptable values.  These sets have values added to them occasionally.

In [3]:
requests.get(f'{adsurl}/core_file_fields').json()

{'archive_filename': 'str',
 'caldat': 'datetime64',
 'date_obs_max': 'datetime64',
 'date_obs_min': 'datetime64',
 'dec_max': 'np.float64',
 'dec_min': 'np.float64',
 'depth': 'np.float64',
 'exposure': 'np.float64',
 'filesize': 'np.int64',
 'ifilter': 'category',
 'instrument': 'category',
 'md5sum': 'str',
 'obs_mode': 'category',
 'obs_type': 'category',
 'original_filename': 'str',
 'proc_type': 'category',
 'prod_type': 'category',
 'proposal': 'category',
 'ra_max': 'np.float64',
 'ra_min': 'np.float64',
 'release_date': 'datetime64',
 'seeing': 'np.float64',
 'site': 'category',
 'survey': 'category',
 'telescope': 'category',
 'updated': 'datetime64'}

### Get Aux File field names and types <a id="aux_file" />
These occur once per FITS file but will be slower to search.  All keywords from the Primary HDU (except those in Core File Fields) are included. All types default to "str", but with input from experts we can refine these.

In [4]:
requests.get(f'{adsurl}/aux_file_fields/decam/raw/').json()

{'AIRMASS': 'str',
 'AOS': 'str',
 'ASTIG1': 'str',
 'ASTIG2': 'str',
 'AZ': 'str',
 'BCAM': 'str',
 'CALMODE': 'str',
 'CAMSHUT': 'str',
 'CHECKVER': 'str',
 'CONSTVER': 'str',
 'DARKTIME': 'str',
 'DATE-OBS': 'str',
 'DEC': 'str',
 'DETSIZE': 'str',
 'DIMM2SEE': 'str',
 'DIMMSEE': 'str',
 'DODX': 'str',
 'DODY': 'str',
 'DODZ': 'str',
 'DOMEAZ': 'str',
 'DOMEFLOR': 'str',
 'DOMEHIGH': 'str',
 'DOMELOW': 'str',
 'DONUTFN1': 'str',
 'DONUTFN2': 'str',
 'DONUTFN3': 'str',
 'DONUTFN4': 'str',
 'DONUTFS1': 'str',
 'DONUTFS2': 'str',
 'DONUTFS3': 'str',
 'DONUTFS4': 'str',
 'DOXT': 'str',
 'DOYT': 'str',
 'DTACCOUN': 'str',
 'DTACQNAM': 'str',
 'DTACQUIS': 'str',
 'DTCALDAT': 'str',
 'DTCOPYRI': 'str',
 'DTINSTRU': 'str',
 'DTNSANAM': 'str',
 'DTOBSERV': 'str',
 'DTPI': 'str',
 'DTPIAFFL': 'str',
 'DTPROPID': 'str',
 'DTQUEUE': 'str',
 'DT_RTNAM': 'str',
 'DTSITE': 'str',
 'DTSTATUS': 'str',
 'DTTELESC': 'str',
 'DTTITLE': 'str',
 'DTUTC': 'str',
 'ERRORS': 'str',
 'EXCLUDED': 'str',
 'EXP

## Advanced File Search of Core and Aux fields, results into Pandas <a id="file_search" />

### Search Files
- Use Database derived field, "ra_min"
- Use _AUX_ field in output and search
- Limit maximum number of results (rows) returned
- Load results into [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html#pandas.DataFrame)

In [12]:
jj = {
    "outfields" : [
        "md5sum",
        "ra_min",  # "ra" is a "range_field" in Postgres, access either part of range
        "archive_filename",
        "instrument",
        "proc_type",
        "obs_type",
        "release_date",
        "proposal",
        "EXPNUM",    # AUX field. Slows search
        "AIRMASS",
    ],
    "search" : [
        ["instrument", "decam"],
        ["proc_type", "raw"],
        ["ra_min",322,323],
        ["EXPNUM", 667000, 669000],
    ]
}
apiurl = f'{adsurl}/fasearch/?limit=10'
print(f'Using API url: {apiurl}')
pd.read_json(json.dumps(requests.post(apiurl,json=jj).json()))

Using API url: https://marsnat1.stage.dm.noao.edu/api/adv_search/fasearch/?limit=10


,AIRMASS,EXPNUM,archive_filename,instrument,md5sum,obs_type,proc_type,proposal,ra_min,release_date,url
0,1.40,667098,/net/archive/mtn/20170814/ct4m/2017B-0951/c4d_...,decam,10ca3658469336b34d836967b9df26c1,object,raw,2017B-0951,322.493042,2017-09-13,https://marsnat1.stage.dm.noao.edu/api/adv_sea...
1,1.41,667099,/net/archive/mtn/20170814/ct4m/2017B-0951/c4d_...,decam,40a3bc0e4812b336e93ef751eb273325,object,raw,2017B-0951,322.493042,2017-09-13,https://marsnat1.stage.dm.noao.edu/api/adv_sea...
2,1.44,667111,/net/archive/mtn/20170814/ct4m/2017B-0951/c4d_...,decam,5174181ec36de019f6ccb563a706fb48,object,raw,2017B-0951,322.493042,2017-09-13,https://marsnat1.stage.dm.noao.edu/api/adv_sea...
3,1.40,667095,/net/archive/mtn/20170814/ct4m/2017B-0951/c4d_...,decam,6b533d96e76490261f37cdf75cfae2db,object,raw,2017B-0951,322.493042,2017-09-13,https://marsnat1.stage.dm.noao.edu/api/adv_sea...
4,1.44,667110,/net/archive/mtn/20170814/ct4m/2017B-0951/c4d_...,decam,928d5dcee69f591aab9251b1b242386b,object,raw,2017B-0951,322.493042,2017-09-13,https://marsnat1.stage.dm.noao.edu/api/adv_sea...
5,1.44,667109,/net/archive/mtn/20170814/ct4m/2017B-0951/c4d_...,decam,a241d847f02ef71d9884173f0d33f851,object,raw,2017B-0951,322.493042,2017-09-13,https://marsnat1.stage.dm.noao.edu/api/adv_sea...
6,1.44,667108,/net/archive/mtn/20170814/ct4m/2017B-0951/c4d_...,decam,b098def5d197da8b6839fa28beff1c18,object,raw,2017B-0951,322.493042,2017-09-13,https://marsnat1.stage.dm.noao.edu/api/adv_sea...
7,1.41,667100,/net/archive/mtn/20170814/ct4m/2017B-0951/c4d_...,decam,c5519e510248cd702d6c46fc71d1b7f7,object,raw,2017B-0951,322.493042,2017-09-13,https://marsnat1.stage.dm.noao.edu/api/adv_sea...
8,1.44,667107,/net/archive/mtn/20170814/ct4m/2017B-0951/c4d_...,decam,cb6d686c7fe9764e01028d9a829c2689,object,raw,2017B-0951,322.493042,2017-09-13,https://marsnat1.stage.dm.noao.edu/api/adv_sea...
9,1.45,667112,/net/archive/mtn/20170814/ct4m/2017B-0951/c4d_...,decam,d8f62f14e40bf25f5f1f9af038927e1a,object,raw,2017B-0951,322.493042,2017-09-13,https://marsnat1.stage.dm.noao.edu/api/adv_sea...


### Find files available for download
You can comment one or the other of the "release_date" search lines to find either public files or proprietary files.
You can retrieve the public files without authentication. But to retrieve the proprietary files you must be an authenticated PI of a proposal associated with the file. Metadata has no proprietary period.

In [6]:
jj = {
    "outfields" : [
        "md5sum",
        "archive_filename",
        "instrument",
        "telescope",
        "proc_type",
        "obs_type",
        "release_date",
        "proposal",
    ],
    "search" : [
        ["release_date", '2005-02-18','2020-01-20'], # public
        #["release_date", '2020-02-18','2020-11-18'], # proprietary
    ]
}
apiurl = f'{adsurl}/fasearch/?limit=5'
print(f'Using API url: {apiurl}')
ads_df = pd.read_json(json.dumps(requests.post(apiurl,json=jj).json()))
ads_df

Using API url: https://marsnat1.stage.dm.noao.edu/api/adv_search/fasearch/?limit=5


,archive_filename,instrument,md5sum,obs_type,proc_type,proposal,release_date,telescope,url
0,/net/archive/pipeline/Q20110409/C4M11A/2011040...,mosaic_2,0000004ab27d9e427bb93c640b358633,object,instcal,2011A-0477,2012-09-28,ct4m,https://marsnat1.stage.dm.noao.edu/api/adv_sea...
1,/net/archive/pipeline/Q20150727/DEC15A/2015072...,decam,0000065ee227de201686b54706ba58e9,object,resampled,2015A-0618,2017-01-13,ct4m,https://marsnat1.stage.dm.noao.edu/api/adv_sea...
2,/net/archive/pipeline/Q20161102/K4N16B/2016101...,newfirm,0000079ed58542c36a059467cc748420,object,instcal,2013B-0236,2018-04-10,kp4m,https://marsnat1.stage.dm.noao.edu/api/adv_sea...
3,/net/archive/mtn/20140824/soar/2014B-0602/psg_...,goodman,0000088eabc78f332e82ccfe7f4c2f8e,object,raw,2014B-0602,2016-02-15,soar,https://marsnat1.stage.dm.noao.edu/api/adv_sea...
4,/net/archive/pipeline/Q20130905/DEC13B/2013081...,decam,00000987d6968004b925030189ad24b9,object,instcal,2013B-0325,2015-02-09,ct4m,https://marsnat1.stage.dm.noao.edu/api/adv_sea...


#### Get a URL that can be used to retrieve a single public FITS file.  
Pasting this in your browser will result in dialog box that lets you save the file locally.

In [7]:
selectedRow=0
ads_df['url'][selectedRow]

'https://marsnat1.stage.dm.noao.edu/api/adv_search/retrieve/0000004ab27d9e427bb93c640b358633/'

# FIX!!! (the working equiv of above is .../api/natica/retrieve/...)
Do we really want the URL column to be present no matter what user asks for?!? Takes up space and they may not want it.

#### This gets no results (no Archive filename ends with "FOOBAR")
The "archive_filename" is a string type so there are many modifiers that can be used with it: 
_exact, iexact, startswith, istartswith, endswith, iendswith, contains, icontains, regex, iregex_


In [8]:
jj = {
    "outfields" : ["archive_filename", "md5sum"],
    "search" : [
        ["archive_filename", "k4m_170308_045842_ori.fits.fz.FOOBAR", 'endswith'],
    ]
}
pd.read_json(json.dumps(requests.post(f'{adsurl}/fasearch/?limit=9',json=jj).json()))

""


### Can we return something better for above when there are zero results? !!!

### It is an error to search using AUX field without limiting to instrument
An error is raised if an AUX field is in the search clause without also searching for and *instrument* and *proc_type*. Uncommenting the *instrument* line below will fix the problem.

In [9]:
jj = {
    "outfields" : [
        "md5sum",
        "archive_filename",
        "instrument",
        "proc_type",
        "EXPNUM",    # AUX field. Slows search
    ],
    "search" : [
        #["instrument", "decam"],  # uncomment this to eliminate error
        ["proc_type", "raw"],
        ["EXPNUM", 667000, 669000],
    ]
}
requests.post(f'{adsurl}/fasearch/?limit=9',json=jj).json()

{'errorMessage': 'No "instrument" was provided when searching for AUX fields. For performance reasons you must also filter on "instrument" and "proc_type" that contain the AUX fields you are searching on (EXPNUM).',
 'errorCode': 'NATERR'}

In [10]:
fileid = ads_df['md5sum'][0]
fileurl = f'{natroot}/api/retrieve/{fileid}'
fileurl

'https://marsnat1.stage.dm.noao.edu/api/retrieve/0000004ab27d9e427bb93c640b358633'

#### Attempt to get File
There are several reasons attempt may fail, so check status!  Most reasons only apply to Proprietary files and include: 
1. There is no proposal associated with the File
1. User is not authenticated
1. User is not a PI or CoPI associated with the proposal

For more details on retrieving proprietary files see the **api-authentication** notebook.

In [11]:
response = requests.get(fileurl)
if response.status_code == 200:
    print(f'HTTP-Status={response.status_code} File-size={len(response.content):,}')    
    zimage = pyfits.getdata(autils.data.download_file(fileurl,cache=True,show_progress=False,timeout=120))
    plt.rcParams["figure.figsize"] = [16,9]
    plt.imshow(zimage,cmap=matplotlib.cm.gray_r)
    zimage.shape
else:
    print(f'content={response.content}')
    print(response.json()['errorMessage'])
    print(response.json()['traceback'])  # for API developer use


content=b'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta http-equiv="content-type" content="text/html; charset=utf-8">\n  <title>Page not found at /api/retrieve/0000004ab27d9e427bb93c640b358633</title>\n  <meta name="robots" content="NONE,NOARCHIVE">\n  <style type="text/css">\n    html * { padding:0; margin:0; }\n    body * { padding:10px 20px; }\n    body * * { padding:0; }\n    body { font:small sans-serif; background:#eee; color:#000; }\n    body>div { border-bottom:1px solid #ddd; }\n    h1 { font-weight:normal; margin-bottom:.4em; }\n    h1 span { font-size:60%; color:#666; font-weight:normal; }\n    table { border:none; border-collapse: collapse; width:100%; }\n    td, th { vertical-align:top; padding:2px 3px; }\n    th { width:12em; text-align:right; color:#666; padding-right:.5em; }\n    #info { background:#f6f6f6; }\n    #info ol { margin: 0.5em 4em; }\n    #info ol li { font-family: monospace; }\n    #summary { background: #ffc; }\n    #explanation { background:#eee; bor

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Get Core HDU field names and types <a id="core_hdu" />
These occur once per non-primary HDU and are optimized for fast search.

In [ ]:
requests.get(f'{adsurl}/core_hdu_fields').json()

### Get Aux HDU field names and types <a id="aux_hdu" />
These occur once per non-primary HDU but will be slower to search.  All keywords from the non-primary HDUs (except those in Core HDU Fields) are included.

In [ ]:
requests.get(f'{adsurl}/aux_hdu_fields/decam/raw/').json()

## Advanced HDU Search of Core and Aux fields, results into Pandas <a id="hdu_search" />  

# FIX!!!
- Should protect against AUX field use unless both Instrument and Proc_type are given.
- Need AUX field that can be found in both DEV and PAT

In [ ]:
jj = {
    "outfields" : [
        "fitsfile__archive_filename",
        "fitsfile__caldat",
        "fitsfile__instrument",
        "fitsfile__proc_type",
        "ra",
        "dec",
        "AIRMASS"  # AUX field. Slows search
    ],
    "search" : [
        ["fitsfile__instrument", "decam"],
        #["fitsfile__proc_type", "raw"],
    ]
}
df = None # Pandas DataFrame
apiurl=f'{adsurl}/hasearch/?limit=9'
response = requests.post(apiurl,json=jj)

if response.status_code == 200:
    df = pd.read_json(json.dumps(response.json()))
else:
    print(response.json()['errorMessage'])
    print(response.json()['traceback'])  # for API developer use
df

## Error Conditions <a id="errors" />

### Examples of bad search specification
#### Attempt to search unknown field name

In [ ]:
jj={
    "outfields" : [
        "ifilter",
        "archive_filename"
    ],
    "search" : [
        ["snafu","wubba"], # unknown field "snafu"
        ["instrument", "decam"]
    ]
}
df = None # Pandas DataFrame
apiurl=f'{adsurl}/fasearch/?limit=9'
response = requests.post(apiurl,json=jj)

if response.status_code == 200:
    df = pd.read_json(json.dumps(response.json()))
else:
    print(response.json()['errorMessage'])
    #print(response.json()['traceback'])  # for API developer use
df

#### Attempt to retrieve field name not in known list

In [ ]:
jj={
    "outfields" : [
        "baz",  # unknown field
        "foo",  # unknown field
        "ifilter",
        "archive_filename"
    ],
    "search" : [
        ["ifilter", "GG395 kGG395"],
    ]
}
df = None # Pandas DataFrame
apiurl=f'{adsurl}/fasearch/?limit=9'
response = requests.post(apiurl,json=jj)

if response.status_code == 200:
    df = pd.read_json(json.dumps(response.json()))
else:
    print(response.json()['errorMessage'])
    #print(response.json()['traceback'])  # for API developer use
df



# HERE !!! <a id="here">

In [ ]:
import datetime
elapsed = toc()  # Seconds since start of Notebook load (assuming fresh run)
print(f'Elapsed seconds={elapsed}')
str(datetime.datetime.now())

# Resources and references <a id="doc" />
- [dal API](http://marsnat2.pat.dm.noao.edu:8000/api-docs/#/dal)
We should look at fancy annotation to make this better. It would be better for programmers to have very clear documentation of the types and purposes of all parameters that can be passed to each service

- [Data  Models](https://marsnat1.stage.dm.noao.edu/admin/doc/models/)

- [NOAO Data Handbook](http://ast.noao.edu/sites/default/files/NOAO_DHB_v2.2.pdf) version 2.2, May 2015

# Glossary <a id="glos">
    
- Proc_type: Processing Type; pipeline process used to generate reduced FITS from raw FITS
- Prod_type
- Obs_type
- Obs_mode
    
Term     | Definition
---------|----------
ProcType | Processing Type; pipeline process used to generate reduced FITS from raw FITS